## Setup

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import tarfile
import requests
import json
import numpy as np
from io import BytesIO

## Configuration  
Update the `<api-key>`, `<bucket>` and `<key>` to the correct values for your object storage.  They can be found via the "Insert to code" function in the data panel.

In [ ]:
s3info = {
    'apikey': "<api-key>",
    'endpoint': "https://s3.private.us.cloud-object-storage.appdomain.cloud",
    'bucket' : "<bucket>",
    'key' : "<key>"
}

 Update the `<fhir_user_password>` and `<fhir_server_ip>` based on your OpenShift Deployment and Service configurations

In [3]:
# HTTP request info
from requests.auth import HTTPBasicAuth
httpAuth = HTTPBasicAuth('fhiruser', '<fhir_user_password>')     # Update before running
urlBase = 'https://<fhir_server_ip>:30100/fhir-server/api/v4/'  # Update before running
headers = {'Content-Type': 'application/json'}

## Read JSON patient images from tar archive  
- Set up access to COS
- Read tar object from COS
- Load tar objects to a list

In [5]:
s3client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=s3info['apikey'],
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

In [6]:
input_tar_file = s3client.get_object(Bucket = s3info['bucket'], Key = s3info['key'])

In [7]:
tarContent = input_tar_file['Body'].read()

In [8]:
tf = tarfile.open(fileobj = BytesIO(tarContent))

In [9]:
tarMembers = tf.getmembers()

In [10]:
print(len(tarMembers))

2001


## Load JSON objects to FHIR server
- Loop through tarMembers list of objects
- POST each to FHIR server
- Print return code

In [11]:
# Disable warnings for insecure requests - don't do this in production!
import urllib3
urllib3.disable_warnings()

**NOTE:** - will print line for each entry in tarMembers.  Comment out `print` if you don't want a lot of output.

In [13]:
for x in range (len(tarMembers)):
    content = tf.extractfile(tarMembers[x]).read()
    r = requests.post(urlBase,
        data=content,
        headers=headers,  
        auth=httpAuth,
        verify=False)    
    print(x, tarMembers[x].name, len(content), r.ok, r.status_code)

0 Aaron_Boyer_80f61574-2923-4e69-9027-974674180beb.json 45429 True 200
1 Aaron_Lynch_eb96fa19-9247-9e11-afb6-5cdda593a6bd.json 213647 True 200
2 Abbey_Ullrich_4d6b7751-ba15-fe97-158c-eede1880daf3.json 405422 True 200
3 Abbey_Weimann_add2bd3e-cb47-63b4-85a8-e88ae25e104f.json 439837 True 200
4 Abel_Kris_1f9e02ca-3e89-6d24-f842-a4280ae473c4.json 271435 True 200


## Check how many objects were loaded

In [16]:
def getFHIR(queryParams, urlLoc):
    resp_loc = requests.get(urlBase + urlLoc, 
                    headers=headers,  
                    params=queryParams,
                    auth=httpAuth,
                    verify=False)        
    resp_dict = (json.loads(resp_loc.content))
    return resp_dict

In [17]:
queryParams = {"_count" : 1}

for queryType in ['Location', 'Patient', 'Condition']:
    resp_dict = getFHIR(queryParams, queryType)
    print("Loaded "+ str(resp_dict['total']) + " " + queryType + "s")

Loaded 2090 Locations
Loaded 2104 Patients
Loaded 2486 Conditions
